<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<mb><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>

## 코로나 국가별 확진자 수 추이 분석/시각화

### 날짜별 확진자수 최종 데이터프레임 읽기 (df_confirmed)

In [ ]:
import pandas as pd
df_confirmed = pd.read_csv("COVID-19-master/final_df.csv")
df_confirmed.head()

In [ ]:
df_confirmed.shape

### 국가명과 iso2 매칭 테이블 읽기 (country_info)
- 국기 이미지를 가져오기가 쉽지 않음 (1년에 한번씩 변경하였음)
  - 두글자의 ISO 국가 코드 기반(예: KR), 다음 이미지가 현재 가능함 (이후 또 변경시, 본 주피터 노트북에 업데이트 예정)
  - 예: https://public.flourish.studio/country-flags/svg/kr.svg

#### 이슈
  - csv 파일에서 다음 데이터값은 데이터프레임으로 읽어들일 때, 자동으로 결측치로 변환됨
    - ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, ‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘<NA>’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, ‘n/a’, ‘nan’, ‘null’
    - Namibia 국가의 iso2 값이 NA 이므로 결측치 변환을 막기 위해 다음과 같은 옵션 설정 필요
      - keep_default_na=False : 위 데이터값을 자동으로 결측치로 변환하지 않고, na_values로 지정한 데이터만 결측치로 변환
      - na_values='' : 결측치로 변환할 값을 지정

In [ ]:
country_info = pd.read_csv("COVID-19-master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv", encoding='utf-8-sig', keep_default_na=False, na_values='')
country_info.head()

In [ ]:
country_info[country_info['Country_Region'] == 'Namibia']

In [ ]:
country_info = country_info[['iso2', 'Country_Region']]
country_info.head()

- 중복 행 제거

In [ ]:
country_info.shape

In [ ]:
country_info = country_info.drop_duplicates(subset='Country_Region', keep='last')
country_info.shape

### 날짜별 국가별 확진자수와 국가별 iso2 값 병합

In [ ]:
doc_final_country = pd.merge(df_confirmed, country_info, how='left', on='Country_Region')
doc_final_country.head()

In [ ]:
doc_final_country.shape

#### 없는 데이터(NaN) 확인하기

In [ ]:
doc_final_country.isnull().sum()

### 특정 컬럼에 없는 데이터(NaN)가 있는 경우, 해당 행들만 보여주기
- 특정 컬럼 조건에 맞는 행들만 추출 가능
  - dataframe[dataframe[컬럼명] 조건]

In [ ]:
doc_final_country[doc_final_country['iso2'].isnull()]

In [ ]:
doc_final_country[doc_final_country['iso2'].isnull()]['Country_Region']

| 번호 | 항목                    | 설명                           |
|------|-------------------------|--------------------------------|
| 5    | Antarctica              | 국가가 아닌 남극 대륙            |
| 45   | Cruise Ship             | 국가가 아닌 유람선 관련 데이터   |
| 50   | Diamond Princess        | 국가가 아닌 유람선 (유명 크루즈 선박)|
| 108  | MS Zaandam              | 국가가 아닌 유람선 (크루즈 선박)   |
| 136  | Others                  | 국가 외 기타 항목               |
| 170  | Summer Olympics 2020    | 국가가 아닌 이벤트 (올림픽 대회)  |
| 197  | Winter Olympics 2022    | 국가가 아닌 이벤트 (올림픽 대회)  |


#### 참고: 복합 조건은 
- dataframe[(조건1) & (조건2)] : 조건1과 조건2 모두 만족 (and 조건)
- dataframe[(조건1) | (조건2)] : 조건1 또는 조건2 만족 (or 조건)
- 조건에 괄호를 넣는 것이 오동작을 방지하기 위해 좋음

```
nan_rows = doc_final_country[(doc_final_country['iso2'].isnull()) & (doc_final_country['4/01/2020'] == 0)]
nan_rows.head()
```

### 특정 컬럼에 없는 데이터 삭제하기

In [ ]:
doc_final_country = doc_final_country.dropna(subset=['iso2'])

In [ ]:
doc_final_country[doc_final_country['iso2'].isnull()]

### 국기 링크를 기존 컬럼 기반해서 만들어, 데이터프레임에 붙이기
- 국기 이미지를 가져오기가 쉽지 않음 (1년에 한번씩 변경하였음)
  - 두글자의 ISO 국가 코드 기반(예: KR), 다음 이미지가 현재 가능함 (이후 또 변경시, 본 주피터 노트북에 업데이트 예정)
  - 예: https://public.flourish.studio/country-flags/svg/kr.svg

- 국기 링크 만들기

In [ ]:
iso2= 'AD'
flag_link = 'https://public.flourish.studio/country-flags/svg/' + iso2.lower() + '.svg'
flag_link

#### 참고: 데이터프레임에 신규 컬럼 추가하기
- 데이터 프레임에 신규 컬럼 추가하기
  - 데이터프레임[신규컬럼] = pd.Series(data=데이터)
    - 데이터는 행에 맞게, [0, 1, 2] 와 같은 형태로 넣을 수 있음
- 기존 데이터프레임 컬럼 값을 기반으로 신규 컬럼 추가하기
  - apply() 함수를 사용

In [ ]:
def create_flag_link(column):
    flag_link = 'https://public.flourish.studio/country-flags/svg/' + column.lower() + '.svg'
    print(flag_link)
    return flag_link

doc_final_country['iso2'] = doc_final_country['iso2'].apply(create_flag_link)

### 데이터프레임 컬럼 조정하기

In [ ]:
doc_final_country.head()

#### 1. 데이터프레임에서 필요한 컬럼만 선택하기
- 데이터프레임.columns.tolist() : 컬럼명을 리스트로 변환

<img src="https://davelee-fun.github.io/fixeddata/covid_data_format.png">

In [ ]:
cols = doc_final_country.columns.tolist()
cols.remove('iso2')
cols.insert(1, 'iso2')
cols

#### 내가 정한 조건대로 정렬하기
- **`sorted(..., key=lambda x: pd.to_datetime(x, format='%m/%d/%Y'))`**:  
  - `sorted` 함수는 리스트를 정렬하여 새 리스트를 반환합니다.
  - **`key=lambda x: ...`**:  
    - 각 요소 `x`에 대해 정렬 기준을 지정합니다.
    - `x`는 날짜 형식의 문자열일 것으로 가정됩니다.
  - **`pd.to_datetime(x, format='%m/%d/%Y')`**:  
    - `x`를 지정된 날짜 포맷(`월/일/연도`)에 따라 datetime 객체로 변환합니다.
    - 변환된 datetime 객체를 기준으로 정렬되므로, 날짜 순서대로 정렬됩니다.

In [ ]:
sorted(cols[2:], key=lambda x: pd.to_datetime(x, format='%m/%d/%Y'))

#### 참고: 오름차순 정렬과 내림차순 정렬
- **오름차순 정렬 (낮은 순)**  
  기본적으로 `sorted` 함수는 오름차순 정렬(가장 작은 값부터 큰 값 순)을 수행합니다. 날짜의 경우, 이는 **이른 날짜부터 늦은 날짜** 순으로 정렬됨을 의미합니다.
  
  예시:
  ```python
  # 이른 날짜(낮은 값) -> 늦은 날짜(높은 값) 순
  sorted_list = sorted(cols[2:], key=lambda x: pd.to_datetime(x, format='%m/%d/%Y'))
  ```

- **내림차순 정렬 (높은 순)**  
  내림차순 정렬(큰 값부터 작은 값 순)을 원한다면, `sorted` 함수의 `reverse` 매개변수를 `True`로 설정하면 됩니다. 날짜의 경우, 이는 **늦은 날짜부터 이른 날짜** 순으로 정렬됨을 의미합니다.
  
  예시:
  ```python
  # 늦은 날짜(높은 값) -> 이른 날짜(낮은 값) 순
  sorted_list_desc = sorted(cols[2:], key=lambda x: pd.to_datetime(x, format='%m/%d/%Y'), reverse=True)
  ```

이처럼 `reverse` 매개변수를 통해 날짜 정렬의 순서를 쉽게 조절할 수 있습니다.  
- `reverse=False` 또는 생략 시: 오름차순 (낮은 순)  
- `reverse=True` 시: 내림차순 (높은 순)

#### 리스트 합치기
- 리스트타입 + 리스트타입 으로 리스트를 합칠 수 있음 

In [ ]:
cols = cols[:2] + sorted(cols[2:], key=lambda x: pd.to_datetime(x, format='%m/%d/%Y'))
cols

#### 데이터프레임 컬럼 위치 변경

In [ ]:
doc_final_country = doc_final_country[cols]

In [ ]:
doc_final_country.head()

#### 컬럼명 변경

In [ ]:
cols[1] = 'Country_Flag'

In [ ]:
doc_final_country.columns = cols
doc_final_country.head()

### 최종 가공 완료 파일 저장

In [ ]:
doc_final_country.to_csv("COVID-19-master/final_covid_data_for_graph.csv")

### 그래프 만들기
- https://app.flourish.studio/ 로그인
- New visualization 선택 -> Bar Chart Race 선택 -> 데이터 수정 (파일 업로드)
- 최종 그래프 예: https://public.flourish.studio/visualisation/17998107/

<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>